In [30]:
# A notebook in R Language
# Use Machine Learning to predict the probability of needing Post-Mastectom Radiotherapy
# Author: Yifu (Charles) Chen

### Install packages ###
# Please install the following packages before running this notebook:
# install.packages("standardize")
# install.packages("randomForest")

### Import libraries ###
# Logistic Regression, Random Forest, and XGBoost
library(randomForest)
library(xgboost)
# standardization
library(data.table)
# cross-validation
library(crossval)
# performance metrics
library(performance)
# plotting
library(ggplot2)



ERROR: Error in library(randomForest): there is no package called ‘randomForest’


## Load Dataset

In [26]:

# Directory of DataFrames
dir_to_df <- "/Users/yifu/PycharmProjects/Radiotherapy-Prediction/data/experiments/May18 Datasets for Experiments Table Debugged"
# Assume all files in directory are CSV DataFrame files
paths_to_df <- list.files(dir_to_df, full.names = TRUE)
paths_to_df

# Read each DataFrame file
dfs <- list()
for (path in paths_to_df) {
    temp_df <- read.csv(path, header = TRUE, sep = ",")
    dfs[[path]] <- temp_df
}

ERROR: Error in standardize(df[-c("POS_did_the_patient_receive_pm")]): could not find function "standardize"


## Pre-Process Dataset

In [27]:
# The target variable is named "POS_did_the_patient_receive_pm"
# Standardize the data by removing the mean and scaling to unit variance
for (df in dfs) {
    df <- standardize(df, na.rm = TRUE)
}


# For each DataFrame, standardize 
for (df_path in paths_to_df) {
    # Read the DataFrame
    df <- dfs[[df_path]]
    # Standardize all the DataFrame columns except POS_did_the_patient_receive_pm, with mean 0 and standard deviation 1
    df_std <- standardize(df[-c("POS_did_the_patient_receive_pm")])
    # Read a copy of the DataFrame
    df_copy <- data.frame(df_std)
    # Impute the DataFrame's missing values using KNN Imputation, with K=5
    df_imputed <- knnimpute(df_copy, k = 5)
    # Build a Logistic Regression Model
    lr_model <- glm(POS_did_the_patient_receive_pm ~ ., data =  df_imputed, family = binomial)
    # Build a Random Forest Model
    rf_model <- rf(df$POS_did_the_patient_receive_pm ~ ., data = df_imputed, ntree = 100)
    # Build a XGBoost Model
    xgb_model <- xgb(df$POS_did_the_patient_receive_pm ~ ., data = df_imputed, nround = 100)
    # Save the models to the hashmap
    models_map[[df_path]] <- list(LR = lr_model, RF = rf_model, XGB = xgb_model)
}


ERROR: Error in standardize(df, na.rm = TRUE): could not find function "standardize"


## Build Models

In [ ]:
# Create a hashmap for saving the models for each DataFrame
# Schema: {DataFrame_path_name: {LR: LR_Model, RF: RF_Model, XGB: XGB_Model}}
models_map <- list()

# Iterate through each DataFrame, and build Logistic Regression, Random Forest, and XGBoost Models to predict target column "POS_did_the_patient_receive_pm"
for (df_path in paths_to_df) {
    # Read the DataFrame
    df <- dfs[[df_path]]
    # Standardize all the DataFrame columns except POS_did_the_patient_receive_pm, with mean 0 and standard deviation 1
    df_std <- standardize(df[-c("POS_did_the_patient_receive_pm")])
    # Read a copy of the DataFrame
    df_copy <- data.frame(df_std)
    # Impute the DataFrame's missing values using KNN Imputation, with K=5
    df_imputed <- knnimpute(df_copy, k = 5)
    # Build a Logistic Regression Model
    lr_model <- glm(POS_did_the_patient_receive_pm ~ ., data =  df_imputed, family = binomial)
    # Build a Random Forest Model
    rf_model <- rf(df$POS_did_the_patient_receive_pm ~ ., data = df_imputed, ntree = 100)
    # Build a XGBoost Model
    xgb_model <- xgb(df$POS_did_the_patient_receive_pm ~ ., data = df_imputed, nround = 100)
    # Save the models to the hashmap
    models_map[[df_path]] <- list(LR = lr_model, RF = rf_model, XGB = xgb_model)
}

